In [1]:
# Kütüphaneler
import time
start_time = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image, ImageDraw, ImageFont

print("Kütüphaneler başarıyla yüklendi.")

Kütüphaneler başarıyla yüklendi.


In [2]:
# Yardımcı Fonksiyonlar

def assign_kisim(makina_kodu):
    """
    Makina koduna göre hangi kısıma ait olduğunu belirleyen fonksiyon
    """
    if makina_kodu in kisim_2_1:
        return "KISIM 2.1"
    elif makina_kodu in kisim_2_2:
        return "KISIM 2.2"
    elif makina_kodu in kisim_3_1:
        return "KISIM 3.1"
    elif makina_kodu in kisim_3_2:
        return "KISIM 3.2"
    elif makina_kodu in kisim_4_1:
        return "KISIM 4.1"
    elif makina_kodu in kisim_4_2:
        return "KISIM 4.2"
    elif makina_kodu in kisim_5_1:
        return "KISIM 5.1"
    else:
        return "Diğer"

def stop_time_sum(data_frame):
    """
    Duruş sürelerini toplar ve belirli kategorileri (yemek molası, tasarım duruşları, SMED) gruplar
    """
    toplam_sureler = data_frame.groupby("Duruş Adı")["Süre (Saniye)"].sum().reset_index() 
    toplam_sureler = toplam_sureler.sort_values(by="Süre (Saniye)", ascending=False)

    # Yemek molası ve tasarım duruşlarını filtreleme
    yemek_molasi = toplam_sureler[toplam_sureler['Duruş Adı'].str.contains("YEMEK MOLASI", case=False)]
    tasarim_duruslari = toplam_sureler[toplam_sureler['Duruş Adı'].str.contains("TASARIM", case=False)]
    smed_duruslari = toplam_sureler[toplam_sureler['Duruş Adı'].str.contains("SMED", case=False)]

    # Süreleri toplama
    yemek_molasi_toplam = yemek_molasi['Süre (Saniye)'].sum()
    tasarim_duruslari_toplam = tasarim_duruslari['Süre (Saniye)'].sum()
    smed_duruslari_toplam = smed_duruslari['Süre (Saniye)'].sum()

    # "Yemek Molası" ve "Tasarım Duruşları" için yeni toplam satırları
    new_data = {
        'Duruş Adı': ['YEMEK MOLASI', 'TASARIM DURUŞLARI', 'AYAR'],
        'Süre (Saniye)': [yemek_molasi_toplam, tasarim_duruslari_toplam, smed_duruslari_toplam]
    }
    new_df = pd.DataFrame(new_data)

    # Yemek molası ve tasarım duruşlarının eski satırlarını silme
    toplam_sureler = toplam_sureler[~toplam_sureler['Duruş Adı'].str.contains("YEMEK MOLASI|TASARIM|SMED", case=False)]

    # Güncellenmiş DataFrame ile yeni satırları birleştirme
    toplam_sureler = pd.concat([toplam_sureler, new_df], ignore_index=True)

    # Süreye göre büyükten küçüğe sıralama
    toplam_sureler = toplam_sureler.sort_values(by="Süre (Saniye)", ascending=False).reset_index(drop=True)
    second2minute(toplam_sureler)
    
    return toplam_sureler

def last_row_correction(dataframe, column, kisimlar_dict):
    """
    Verinin son satırını kontrol edip gerekirse temizleyen fonksiyon
    """
    if len(dataframe) == 0:
        return
        
    tezgah = column.iat[-1]
    
    for kisim, tezgah_listesi in kisimlar_dict.items():
        if tezgah in tezgah_listesi:
            çıktı = "Tezgah verisi"
            break
        else:
            çıktı = "Başka bir veri"  # Hiçbir kısımda yoksa
    
    if çıktı == "Başka bir veri":
        dataframe.drop(dataframe.index[-1], inplace=True)

def second2minute(df, second="Süre (Saniye)", minute="Süre (Dakika)"):
    """
    Saniye cinsinden süreleri dakikaya çeviren fonksiyon
    """
    df["Süre (Dakika)"] = (df["Süre (Saniye)"] / 60).astype(int)
    df.drop(columns=["Süre (Saniye)"], inplace=True)

def filter_sort(df, max_week, gozlemlenecek="KISIM"):
    """
    Belirli bir haftaya ait veriyi filtreleyip sıralayan fonksiyon
    """
    # 'KISIM', 'Hafta' ve 'Duruş Adı' kolonlarına göre gruplandırarak 'Süre (Saniye)' kolonunun toplamını hesaplıyoruz
    grouped_df = df.groupby([gozlemlenecek, 'Hafta', 'Duruş Adı'])['Süre (Saniye)'].sum().reset_index()
    grouped_df = grouped_df[grouped_df[gozlemlenecek] != 'Diğer']
    
    # 'Duruş Adı' değerlerini yeniden sınıflandırıyoruz
    grouped_df['Duruş Adı'] = grouped_df['Duruş Adı'].apply(lambda x: 'YEMEK MOLASI' if 'YEMEK MOLASI' in x else ('TASARIM DURUŞLARI' if 'TASARIM' in x else x))
    
    # Aynı 'KISIM', 'Hafta' ve 'Duruş Adı' altında gruplandırıp, 'Süre (Dakika)' kolonunu topluyoruz
    grouped_df = grouped_df.groupby([gozlemlenecek, 'Hafta', 'Duruş Adı'], as_index=False)['Süre (Saniye)'].sum()
    
    # Maksimum hafta değerine göre filtreleme yapıyoruz
    latest_week_df = grouped_df[grouped_df['Hafta'] == max_week]
    
    top10 = (
        latest_week_df.sort_values([gozlemlenecek, 'Süre (Saniye)'], ascending=[True, False])
        .groupby(gozlemlenecek)
        .head(10)
        .reset_index(drop=True)
    )
    
    # top_10_duruslar veri setindeki KISIM ve Duruş Adı sütunlarına göre grouped_df'yi filtrele
    filtered_df = grouped_df.merge(top10[[gozlemlenecek, 'Duruş Adı']], on=[gozlemlenecek, 'Duruş Adı'])
    # Veriyi Süre (Dakika) sütununa göre azalan sırada sıralama
    filtered_df = filtered_df.sort_values(by='Süre (Saniye)', ascending=False)  
    second2minute(filtered_df)
    
    return filtered_df

def combine_images_horizontal(image1_path, image2_path, output_path):
    """
    İki görseli yatay olarak birleştiren fonksiyon
    """
    # Resimleri yükle
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    
    # Yeni resmin genişliği ve yüksekliği
    new_width = image1.width + image2.width
    new_height = max(image1.height, image2.height)
    
    # Yeni bir boş resim oluştur
    combined_image = Image.new("RGB", (new_width, new_height))
    
    # Resimleri birleştir
    combined_image.paste(image1, (0, 0))
    combined_image.paste(image2, (image1.width, 0))
    
    # Yeni resmi kaydet
    combined_image.save(output_path)

print("Yardımcı fonksiyonlar tanımlandı.")

Yardımcı fonksiyonlar tanımlandı.


In [3]:
# Görselleştirme Fonksiyonları

def means2png(title, oee, performans, kullanılabilirlik, kalite, path):
    """
    OEE, performans, kullanılabilirlik ve kalite değerlerinin görselleştirilmesini sağlayan fonksiyon
    """
    # Yuvarlama
    try:
        oee = round(oee * 100)
        performans = round(performans * 100)
        kullanılabilirlik = round(kullanılabilirlik * 100)
        kalite = round(kalite * 100)
    except:
        oee, performans, kullanılabilirlik, kalite = "sayısal", "bir", "veri", "yoktur"
    
    # Görsel boyutları
    width, height = 230, 240

    # Görsel oluştur
    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    # Font yükleme
    try:
        font_large = ImageFont.truetype("arial.ttf", 30)
        font_medium = ImageFont.truetype("arial.ttf", 12)
    except:
        # Sistem fontlarını kullanma
        font_large = ImageFont.load_default()
        font_medium = ImageFont.load_default()

    # Metin çizme
    draw.text((20, 15), title, fill="black", font=font_large)
    draw.text((20, 70), f"TOPLAM TEE\nORANI : %{oee}", fill="black", font=font_large, spacing=5)
    draw.text((20, 175), f"PERFORMANS ORANI : %{performans}\nKULLANILABİLİRLİK ORANI : %{kullanılabilirlik}\nKALİTE ORANI : %{kalite}",
              fill="black", font=font_medium, spacing=5)

    # Dosya yolunu oluşturma
    full_path = os.path.join("Raporlar/Tee/", path)
    directory = os.path.dirname(full_path)
    
    # Dizin yoksa oluştur
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Görseli kaydet
    image.save(full_path)

# 1. visualize fonksiyonunu düzelt:
def visualize(data, threshold=2, baslik="Pasta Grafik"):    
    """
    Pasta grafiği oluşturan fonksiyon
    """
    if "KISIM" in baslik:
        folder_path = 'Raporlar/Kısımlar/Son Hafta'
    else:
        folder_path = 'Raporlar/Genel'
    
    os.makedirs(folder_path, exist_ok=True)
    
    total_time = data["Süre (Dakika)"].sum()
    data["Yüzde"] = (data["Süre (Dakika)"] / total_time * 100)
    
    # %threshold'tan küçük olanları "Diğer" olarak grupla
    diger_sure = data[data["Yüzde"] < threshold]["Süre (Dakika)"].sum()
    diger_df = data[data["Yüzde"] >= threshold]
    
    # "Diğer" satırını yeni DataFrame'e ekle
    if diger_sure > 0:
        diger_df = pd.concat([diger_df, pd.DataFrame({"Duruş Adı": ["Diğer"], "Süre (Dakika)": [diger_sure]})])
    
    # Pasta grafik oluşturma
    plt.figure(figsize=(12, 12))
    myexplode = [0.1] * len(diger_df)  # Patlatma ayarları
    
    # Pasta grafiği
    wedges, texts, autotexts = plt.pie(
        diger_df["Süre (Dakika)"], 
        wedgeprops=dict(width=0.7),
        autopct=lambda p: '{:.1f}%'.format(p) if p > threshold else '',  # threshold'tan küçük yüzdeleri göstermeme
        startangle=90,
        colors=sns.color_palette("Pastel2", len(diger_df)),
        explode=myexplode,
        shadow=True
    )
    
    # Wedge'lar üzerinde kutucuklarla bilgi ekleme
    for i, wedge in enumerate(wedges):
        ang = (wedge.theta2 - wedge.theta1) / 2. + wedge.theta1
        y = np.sin(np.deg2rad(ang)) * 1.2
        x = np.cos(np.deg2rad(ang)) * 1.6
    
        # Ok çizme
        plt.annotate('', xy=(x*0.75, y*0.95), xytext=(np.cos(np.deg2rad(ang)) * 0.9, np.sin(np.deg2rad(ang)) * 0.9),
                     arrowprops=dict(arrowstyle='->', color='black', lw=1))
    
        # Bilgi kutucuğu ekleme
        plt.text(x, y, f"{diger_df.iloc[i, 0]}: {diger_df['Süre (Dakika)'].iloc[i]} dk", ha='center', fontsize=10,
                 bbox=dict(facecolor='white', alpha=0.5, edgecolor='blue', boxstyle='round,pad=0.5'))
    
    plt.title(f"{baslik} Toplam Süre (Dakika)", fontsize=14, pad=15)
    plt.axis('equal')  # Eşit oranlar için
    plt.savefig(os.path.join(folder_path, f"{baslik}.png"), dpi=300, bbox_inches='tight')
    plt.close()



def visualize_2(df, gozlem="KISIM", egiklik=75, palet="tab20"):
    """
    4 haftalık karşılaştırma için bar grafiği oluşturan fonksiyon
    """
    if gozlem == "KISIM":
        # Klasör yolunu tanımlama
        folder_path = 'Raporlar/Kısımlar/4 haftalık'  # Klasör adı
    else:
        folder_path = 'Raporlar/Tezgahlar/4 haftalık'
    
    os.makedirs(folder_path, exist_ok=True)  # Klasör yoksa oluştur
    
    # Her gözlem değeri üzerinde döngü
    for gozlemlenen, data in df.groupby(gozlem):
        plt.figure(figsize=(14, 8))
        ax = sns.barplot(data=data, x='Duruş Adı', y='Süre (Dakika)', hue='Hafta', palette=palet)
    
        # Y-ekseni üst sınırını genişletme
        max_height = max([p.get_height() for p in ax.patches])
        ax.set_ylim(0, max_height * 1.1)  # Y-ekseni üst sınırını %10 arttır
    
        # Çubukların üstüne dik metin eklemek
        for i, p in enumerate(ax.patches):
            # Her çubuğa farklı bir ofset ekleyerek konumlandırma
            offset = (i % data['Hafta'].nunique()) * 5  # Haftaların sayısına göre ofset belirleme
            height = p.get_height()
            # Yüksekliği sıfırdan büyük olan çubuklar için metin ekle
            if height > 0:
                ax.annotate(f'{height:.0f}', 
                            (p.get_x() + p.get_width() / 2, height + offset), 
                            ha='center', va='bottom', 
                            fontsize=10, color='black', 
                            rotation=egiklik,  # Metni dik yazdırmak için belirtilen açı
                            xytext=(0, 5), textcoords='offset points')
    
        plt.title(f'{gozlemlenen} - 4 Haftalık En Büyük 10 Duruş Karşılaştırması')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Hafta')
        plt.tight_layout()
        plt.savefig(os.path.join(folder_path, f"{gozlemlenen} - 4 HAFTALIK.png"), dpi=300, bbox_inches='tight')
        plt.close()

def visualize_bar(data, colors="Accent", bundan=None, buna=None, text=1, baslik="Tüm İş Merkezleri"):
    """
    Bar grafiği oluşturan fonksiyon
    """
    # Toplam süreyi hesapla
    total_time = data["Süre (Dakika)"].sum()
    
    # Bar grafiği oluşturma
    plt.figure(figsize=(12, 8))

    if " (Tezgah Başına)" in baslik:
        # Klasör yolunu tanımlama
        folder_path = 'Raporlar/Kısımlar/Son Hafta Tezgah Başına Ortalama'  # Klasör adı
    else:
        folder_path = 'Raporlar/Genel'
    os.makedirs(folder_path, exist_ok=True)  # Klasör yoksa oluştur
    
    # Veri kırpma
    if bundan is not None:
        if bundan < 0:
            data_subset = data.iloc[bundan:].copy()
        else:
            if buna is not None:
                data_subset = data.iloc[bundan:buna].copy()
            else:
                data_subset = data.iloc[bundan:].copy()
    else:
        data_subset = data.copy()
    
    colors = sns.color_palette(colors, len(data_subset))
    
    # Yüzde hesaplama
    data_subset["Yüzde"] = (data_subset["Süre (Dakika)"] / total_time) * 100
    
    # Bar grafiği çizdir
    bars = plt.bar(data_subset["İş Merkezi Kodu "], data_subset["Süre (Dakika)"], color=colors)
    
    if text:
        # Yüzdeleri her bir barın üstüne ekleme
        for bar in bars:
            height = bar.get_height()
            percentage = (height / total_time) * 100  # Yüzde hesaplama
            plt.text(
                bar.get_x() + bar.get_width() / 2, height, 
                f"{height} ({percentage:.1f}%)",  # Hem süre hem de yüzde gösterme
                ha='center', va='bottom', fontsize=10, color='black', weight='bold'
            )
    
    # Grafik başlık ve eksen etiketleri
    plt.title(f"{baslik}", fontsize=14, pad=15)
    plt.xlabel("İş Merkezi Kodu", fontsize=12)
    plt.ylabel("Süre (Dakika)", fontsize=12)
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(folder_path, f"{baslik}.png"), dpi=300, bbox_inches='tight')
    plt.close()

def plot_bar_and_pie(df, machine_code_column="İş Merkezi Kodu ", stoppage_column="Duruş Adı", 
                    duration_column="Süre (Dakika)", threshold=3, use_pie=False):
    """
    Tezgah bazında çubuk veya pasta grafikleri oluşturan fonksiyon
    """
    # Unique makine kodlarını al
    machine_codes = df[machine_code_column].unique()

    # Klasör yolunu tanımlama
    folder_path = 'Raporlar/Tezgahlar/Son Hafta'
    os.makedirs(folder_path, exist_ok=True)  # Klasör yoksa oluştur
    
    for code in machine_codes:
        # Mevcut makine için verileri filtrele
        machine_data = df[df[machine_code_column] == code]
        
        # Bu makine için veri yoksa atla
        if len(machine_data) == 0:
            continue
            
        total_duration = machine_data[duration_column].sum()
        
        # Toplam süre sıfırsa atla
        if total_duration == 0:
            continue
            
        machine_data.loc[:, "Yüzde"] = (machine_data[duration_column] / total_duration) * 100
        
        # Eşik altındaki değerleri grupla
        other_rows = machine_data[machine_data["Yüzde"] < threshold]
        other_duration = other_rows[duration_column].sum()
        filtered_data = machine_data[machine_data["Yüzde"] >= threshold]
        
        # Eşik altında değerler varsa "Diğer" satırı ekle
        if other_duration > 0:
            other_df = pd.DataFrame({
                machine_code_column: [code],
                stoppage_column: ["Diğer"],
                duration_column: [other_duration],
                "Yüzde": [(other_duration / total_duration) * 100]
            })
            filtered_data = pd.concat([filtered_data, other_df], ignore_index=True)
        
        # Süreye göre azalan sıralama yap
        filtered_data = filtered_data.sort_values(by=duration_column, ascending=False)
        
        if use_pie:
            # PASTA GRAFİK GÖRSELLEŞTİRME
            plt.figure(figsize=(10, 8))
            
            # Pasta grafik oluştur
            wedges, texts, autotexts = plt.pie(
                filtered_data[duration_column],
                labels=None,  # Pasta üzerinde etiket gösterme
                autopct='',  # Boş string vererek üçüncü değeri döndürmesini sağla
                startangle=90,
                colors=sns.color_palette("Pastel1", len(filtered_data)),
                wedgeprops=dict(width=0.5, edgecolor='w'),  # Genişlik <1 halka grafik oluşturur
                pctdistance=0.85
            )
            
            # Yüzdeler ve sürelerle gösterge oluştur
            labels = [f"{row[stoppage_column]}: {row[duration_column]:.0f} dk ({row['Yüzde']:.1f}%)" 
                    for _, row in filtered_data.iterrows()]
            plt.legend(wedges, labels, title="Duruş Nedenleri", 
                      loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
            
            # Halka grafik için merkez daire ekle
            circle = plt.Circle((0, 0), 0.25, fc='white')
            plt.gca().add_artist(circle)
            
            # Merkeze başlık ekle
            plt.text(0, 0, f"{code}\nDuruş\nSüreleri", 
                    ha='center', va='center', fontsize=12, fontweight='bold')
            
        else:
            # ÇUBUK GRAFİK GÖRSELLEŞTİRME
            plt.figure(figsize=(12, 8))
            
            # Dinamik pastel renk paleti
            pastel_colors = sns.color_palette("pastel", n_colors=len(filtered_data))
            
            # Uygun aralıklarla çubuk grafik oluştur
            bars = plt.bar(
                x=range(len(filtered_data)),
                height=filtered_data[duration_column],
                color=pastel_colors,
                width=0.7
            )
            
            # X-ekseni etiketlerini uygun döndürme ile ayarla
            plt.xticks(
                range(len(filtered_data)), 
                filtered_data[stoppage_column],
                rotation=45,
                ha='right'
            )
            
            # Açıklamalar için üstte boşluk bırak
            max_height = max(filtered_data[duration_column]) if len(filtered_data) > 0 else 0
            plt.ylim(0, max_height * 1.2)  # Üstte %20 daha fazla boşluk ekle
            
            # Süre ve yüzde açıklamalarını ekle
            for i, bar in enumerate(bars):
                height = bar.get_height()
                percentage = filtered_data["Yüzde"].iloc[i]
                
                # Sadece anlamlı değer varsa metin ekle
                if height > 0:
                    plt.text(
                        i, height + (max_height * 0.02),  # Çubuğun hemen üzerine konumlandır
                        f'{height:.0f} dk\n({percentage:.1f}%)', 
                        ha='center', va='bottom',
                        fontsize=9, fontweight='bold'
                    )
            
            # Üst ve sağ kenarlıkları kaldır
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            
            # Daha iyi okunabilirlik için grid çizgileri ekle
            plt.grid(axis='y', linestyle='--', alpha=0.3)
            
            # Etiketleri ekle
            plt.xlabel("Duruş Adı", fontsize=12)
            plt.ylabel("Süre (Dakika)", fontsize=12)
        
        # Ortak başlık ve kaydetme ayarları
        plt.title(f"{code} - Duruş Süreleri", fontsize=16, pad=20)
        plt.tight_layout()
        
        # Çubuk ve pasta grafikleri için farklı dosya adları oluştur
        chart_type = "pasta" if use_pie else "cubuk"
        filename = os.path.join(folder_path, f"{code}_{chart_type}_grafik.png")
        
        # Yüksek çözünürlükle kaydet
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()

def create_all_charts(df, machine_column="İş Merkezi Kodu ", stoppage_column="Duruş Adı", 
                     duration_column="Süre (Dakika)", threshold=3):
    """
    Tüm tezgahlar için hem çubuk hem pasta grafikleri oluşturan fonksiyon
    """
    # Çubuk grafikleri oluştur
    plot_bar_and_pie(df, machine_column, stoppage_column, duration_column, threshold, use_pie=False)
    
    # Pasta grafikleri oluştur
    plot_bar_and_pie(df, machine_column, stoppage_column, duration_column, threshold, use_pie=True)
    
    print(f"Toplam {len(df[machine_column].unique())} makine için grafikler oluşturuldu")


In [4]:
# Değişkenler

# Tezgah ve kısım bilgileri
kisim_2_1 = [
    "CT.D01", "CT.D02", "CT.D03", "CT.D04", "CT.D05", "CT.D06", "CT.D07", "CT.D08", 
    "CT.D09", "CT.D10", "CT.D11", "CT.D12", "CT.G1", "CT.KO1", "CT.KO2"
]

kisim_2_2 = [
    "İM.K01", "İM.K03", "İM.MK1", "İM.T01", "İM.T02", "İM.V01"
]

kisim_3_1 = [
    "İM.MV1", "İM.MV2", "İM.MV3", "İM.MV4", "İM.MV5", "İM.MV6", "İM.S01", "İM.S02"
]

kisim_3_2 = [
    "İM.OM01", "İM.OM02", "İM.OM03", "İM.OM04", "İM.OM05", "İM.OM06", 
    "İM.OM07", "İM.OM08", "İM.OM09", "İM.OM10"
]

kisim_4_1 = [
    "İM.M1", "İM.M2", "İM.M3", "İM.M4", "İM.M5", "İM.M6", "İM.M7", "İM.M8", "İM.M9", "İM.O05"
]

kisim_4_2 = [
    "İM.O01", "İM.O02", "İM.O03", "İM.O04", "İM.O06", "İM.O07", "İM.O08", "İM.O09", "İM.O10", "İM.O11"
]

kisim_5_1 = [
    "T.B01", "T.J01", "T.J02", "T.K01", "T.S01", "T.S02", "T.S03", "T.S04", "T.S05", "T.S06", "T.S07", "T.S08"
]

# Tezgah listelerini bir dictionary ile birleştirelim
kisimlar_dict = {
    "KISIM 2.1": kisim_2_1,
    "KISIM 2.2": kisim_2_2,
    "KISIM 3.1": kisim_3_1,
    "KISIM 3.2": kisim_3_2,
    "KISIM 4.1": kisim_4_1,
    "KISIM 4.2": kisim_4_2,
    "KISIM 5.1": kisim_5_1
}

# Tezgah sayısı
kisim_tezgah_sayilari = {kisim: len(tezgahlar) for kisim, tezgahlar in kisimlar_dict.items()}

# Kısımların tezgah sayılarını yazdır
for kisim, sayi in kisim_tezgah_sayilari.items():
    print(f"{kisim}: {sayi} tezgah")

print("Değişkenler tanımlandı.")

KISIM 2.1: 15 tezgah
KISIM 2.2: 6 tezgah
KISIM 3.1: 8 tezgah
KISIM 3.2: 10 tezgah
KISIM 4.1: 10 tezgah
KISIM 4.2: 10 tezgah
KISIM 5.1: 12 tezgah
Değişkenler tanımlandı.


In [5]:
# Veri İçe Aktarma ve Temizleme

# 4 Haftalık Duruş verilerini içeri aktar
file_path = "4 Haftalık Duruş.xlsx"
four_week_df = pd.read_excel(file_path, usecols=["İş Merkezi Kodu ", "Duruş Adı", "Duruş Başlangıç Tarih", "Duruş Bitiş Tarih"])
print(f"4 Haftalık duruş verileri yüklendi. Toplam {len(four_week_df)} satır veri var.")

# Son satır kontrolü - 5 kez kontrol et
for _ in range(1, 6):
    last_row_correction(four_week_df, four_week_df["İş Merkezi Kodu "], kisimlar_dict)

# Tarih sütunlarını datetime formatına dönüştürme
four_week_df['Duruş Başlangıç Tarih'] = pd.to_datetime(four_week_df['Duruş Başlangıç Tarih'])
four_week_df['Duruş Bitiş Tarih'] = pd.to_datetime(four_week_df['Duruş Bitiş Tarih'])
max_baslangic_tarihi = four_week_df['Duruş Başlangıç Tarih'].max()
print(f"En son duruş başlangıç tarihi: {max_baslangic_tarihi}")

# Süreler hesaplanıyor
four_week_df['Süre (Saniye)'] = (four_week_df['Duruş Bitiş Tarih'] - four_week_df['Duruş Başlangıç Tarih']).dt.total_seconds().astype(int)
four_week_df['Süre (Dakika)'] = (four_week_df['Süre (Saniye)'] / 60).astype(int)

# Günlük Çalışma Sürelerini içe aktar
file_path = "Günlük Çalışma Süreleri.xlsx"
calisma = pd.read_excel(file_path, usecols=["Makina Kodu", "Tarih","Çalışma Zamanı","Planlı Duruş",
                                           "Plansız Duruş", "Oee", "Performans", "Kullanılabilirlik", "Kalite"])
print(f"Günlük çalışma süreleri yüklendi. Toplam {len(calisma)} satır veri var.")

# Son satır kontrolü - 5 kez kontrol et
for _ in range(1, 6):
    last_row_correction(calisma, calisma["Makina Kodu"], kisimlar_dict)

# Çalışma sürelerini hesapla
calisma['Süre (Dakika)'] = calisma['Çalışma Zamanı'] - calisma['Planlı Duruş'] - calisma['Plansız Duruş']

# Veriler gruplanıyor ve hazırlanıyor
calisma = (
    calisma.groupby(['Makina Kodu', 'Tarih'], as_index=False)
    .agg(
        Süre_Dakika=('Süre (Dakika)', 'sum'),
        Oee=('Oee', 'mean'),
        Performans=('Performans', 'mean'),
        Kullanılabilirlik=('Kullanılabilirlik', 'mean'),
        Kalite=('Kalite', 'mean'),
    )
)

# Sütun isimlerini düzenleme
calisma = calisma.rename(columns={'Tarih': 'Duruş Başlangıç Tarih', "Makina Kodu": "İş Merkezi Kodu ", "Süre_Dakika": "Süre (Dakika)"})
calisma['Süre (Dakika)'] = calisma['Süre (Dakika)'].astype(int)
calisma["Duruş Adı"] = "ÇALIŞMA SÜRESİ"

# Tarih formatlarını eşitle
four_week_df['Duruş Başlangıç Tarih'] = pd.to_datetime(four_week_df['Duruş Başlangıç Tarih'])
calisma['Duruş Başlangıç Tarih'] = pd.to_datetime(calisma['Duruş Başlangıç Tarih'])

# Eğer görselleştirme için date sürümüne ihtiyacınız olursa, 
# başka bir sütuna kaydedebilirsiniz (isteğe bağlı)
four_week_df['Duruş Başlangıç Tarihi_Date'] = four_week_df['Duruş Başlangıç Tarih'].dt.date
calisma['Duruş Başlangıç Tarihi_Date'] = calisma['Duruş Başlangıç Tarih'].dt.date

# Veri filtreleme - sadece ortak tarih ve makineleri al
matching_dates = four_week_df[['Duruş Başlangıç Tarih']].drop_duplicates()
matching_machine = four_week_df[['İş Merkezi Kodu ']].drop_duplicates()

calisma = calisma[calisma['Duruş Başlangıç Tarih'].isin(matching_dates['Duruş Başlangıç Tarih'])]
calisma = calisma[calisma['İş Merkezi Kodu '].isin(matching_machine['İş Merkezi Kodu '])]

# İki veri setini birleştir
four_week_df = pd.concat([four_week_df, calisma], ignore_index=True)

# Sıfır veya eksik süreleri temizle
four_week_df = four_week_df[(four_week_df['Süre (Dakika)'] != 0) & (four_week_df['Süre (Dakika)'].notna())]

# Çalışma süresi için eksik saniye değerleri doldur
four_week_df.loc[(four_week_df['Duruş Adı'] == 'ÇALIŞMA SÜRESİ') & (four_week_df['Süre (Saniye)'].isna()), 'Süre (Saniye)'] = four_week_df['Süre (Dakika)'] * 60

# Arızalı tezgahları dosyadan oku ve veriden çıkar
try:
    with open("Arızalı Tezgahlar.txt", 'r') as dosya:
        arizali_tezgahlar = dosya.read().split("\n")
    print(f"Arızalı tezgahlar dosyası okundu: {len(arizali_tezgahlar)} tezgah arızalı.")
except:
    print("Arızalı tezgahlar dosyası bulunamadı. Arızalı tezgah yok olarak devam ediliyor.")
    arizali_tezgahlar = []

# Arızalı tezgahları veri kümesinden ve tezgah listelerinden çıkar
if arizali_tezgahlar:
    four_week_df = four_week_df[~four_week_df["İş Merkezi Kodu "].isin(arizali_tezgahlar)]
    
    # Arızalı tezgahları her kısımdan çıkar ve tezgah sayılarını güncelle
    for kisim, tezgahlar in kisimlar_dict.items():
        kisimlar_dict[kisim] = [tezgah for tezgah in tezgahlar if tezgah not in arizali_tezgahlar]
        kisim_tezgah_sayilari[kisim] = len(kisimlar_dict[kisim])
    
    print("Arızalı tezgahlar çıkarıldıktan sonra güncel durum:")
    for kisim, sayi in kisim_tezgah_sayilari.items():
        print(f"{kisim}: {sayi} tezgah")

print(f"Veri hazırlama tamamlandı. Toplam {len(four_week_df)} satır veri var.")

4 Haftalık duruş verileri yüklendi. Toplam 251625 satır veri var.
En son duruş başlangıç tarihi: 2025-03-01 07:58:17.863000
Günlük çalışma süreleri yüklendi. Toplam 3258 satır veri var.
Arızalı tezgahlar dosyası okundu: 1 tezgah arızalı.
Arızalı tezgahlar çıkarıldıktan sonra güncel durum:
KISIM 2.1: 15 tezgah
KISIM 2.2: 6 tezgah
KISIM 3.1: 8 tezgah
KISIM 3.2: 10 tezgah
KISIM 4.1: 10 tezgah
KISIM 4.2: 10 tezgah
KISIM 5.1: 12 tezgah
Veri hazırlama tamamlandı. Toplam 108457 satır veri var.


In [6]:
# Veri Analizi ve Hazırlama

# Kısımları ekleme
four_week_df['KISIM'] = four_week_df['İş Merkezi Kodu '].apply(assign_kisim)

# Tarih bazında haftalık gruplama
four_week_df['Hafta'] = four_week_df['Duruş Başlangıç Tarih'].dt.isocalendar().week

# Her hafta için maksimum tarihi bulalım (haftanın son günü)
hafta_tarihleri = four_week_df.groupby('Hafta')['Duruş Başlangıç Tarih'].max().reset_index()
hafta_tarihleri = hafta_tarihleri.sort_values('Duruş Başlangıç Tarih', ascending=False)

# Tarihe göre sıralanmış hafta numaralarını alalım
weeks = hafta_tarihleri['Hafta'].tolist()

# Haftaları ayrı değişkenlere atama
if len(weeks) >= 4:
    fourth_week = weeks[0]  # En son hafta
    third_week = weeks[1]
    second_week = weeks[2]
    first_week = weeks[3]  # En eski hafta
else:
    print("UYARI: 4 haftadan az veri bulundu!")
    # Eksik hafta değerlerini None veya dummy değerlerle doldur
    available_weeks = len(weeks)
    week_vars = [fourth_week, third_week, second_week, first_week] = [None] * 4
    for i in range(available_weeks):
        week_vars[i] = weeks[i]

print(f"Analiz edilen haftalar: {weeks}")

# TEE (OEE) değerlerini hesapla ve görsellerini oluştur
print("Haftalık TEE (OEE) değerleri hesaplanıyor...")

for week in weeks:
    week_df = four_week_df[four_week_df["Hafta"] == week]
    
    # Genel değerler
    oee4general = week_df["Oee"].mean()
    performans4general = week_df["Performans"].mean()
    kullanılabilirlik4general = week_df["Kullanılabilirlik"].mean()
    kalite4general = week_df["Kalite"].mean()
    
    # Genel görseli oluştur
    path = f"Genel/{week} Hafta.png"
    title = f"{week}. Hafta"
    means2png(title=title, oee=oee4general, performans=performans4general,
              kullanılabilirlik=kullanılabilirlik4general, kalite=kalite4general, path=path)
    
    # Kısım bazında görseller
    for kisim in week_df["KISIM"].unique():
        if kisim != "Diğer":
            kisim_df = week_df[week_df["KISIM"] == kisim]
            oee4part = kisim_df["Oee"].mean()
            performans4part = kisim_df["Performans"].mean()
            kullanılabilirlik4part = kisim_df["Kullanılabilirlik"].mean()
            kalite4part = kisim_df["Kalite"].mean()
            
            path = f"Kısımlar/{week} Hafta/{kisim}.png"
            title = f"{kisim}"
            means2png(title=title, oee=oee4part, performans=performans4part, 
                     kullanılabilirlik=kullanılabilirlik4part, kalite=kalite4part, path=path)
    
    # Tezgah bazında görseller
    for machine in week_df["İş Merkezi Kodu "].unique():
        machine_df = week_df[week_df["İş Merkezi Kodu "] == machine]
        oee4machine = machine_df["Oee"].mean()
        performans4machine = machine_df["Performans"].mean()
        kullanılabilirlik4machine = machine_df["Kullanılabilirlik"].mean()
        kalite4machine = machine_df["Kalite"].mean()
        
        path = f"Tezgahlar/{week} Hafta/{machine}.png"
        title = f"{machine}"
        means2png(title=title, oee=oee4machine, performans=performans4machine,
                 kullanılabilirlik=kullanılabilirlik4machine, kalite=kalite4machine, path=path)

# Son iki haftayı yatay birleştir
try:
    image1_path = f"Raporlar/Tee/Genel/{third_week} Hafta.png"
    image2_path = f"Raporlar/Tee/Genel/{fourth_week} Hafta.png"
    output_path = f"Raporlar/Tee/Genel/{third_week}-{fourth_week} Hafta.png"
    
    combine_images_horizontal(image1_path, image2_path, output_path)
    print(f"{third_week}. ve {fourth_week}. hafta görselleri birleştirildi.")
except Exception as e:
    print(f"Görsel birleştirme sırasında hata oluştu: {e}")

# Son haftanın verilerini ayrı al
max_week = fourth_week if fourth_week is not None else weeks[0]
latest_week_df = four_week_df[four_week_df['Hafta'] == max_week]
latest_week_df.to_excel('Son Hafta için Analiz Edilen Veriler.xlsx', index=False)

print(f"Son hafta ({max_week}) verileri excel dosyasına kaydedildi.")

# Toplam duruş sürelerini hesapla (Yemek, Tasarım ve SMED duruşlarını birleştirerek)
toplam_sureler = stop_time_sum(latest_week_df)
print("Toplam duruş süreleri hesaplandı.")

# Kısım başına ortalama duruş süreleri (tezgah başına düşen)
tezgah_basina_kisim_sureleri = latest_week_df[latest_week_df["Duruş Adı"] != "ÇALIŞMA SÜRESİ"]
tezgah_basina_kisim_sureleri = tezgah_basina_kisim_sureleri.groupby("KISIM")["Süre (Saniye)"].sum().reset_index()
tezgah_basina_kisim_sureleri = tezgah_basina_kisim_sureleri.sort_values(by="KISIM", ascending=True)

# Kısım değerlerini tezgah sayılarına bölerek güncelleme
tezgah_basina_kisim_sureleri["Süre (Saniye)"] = tezgah_basina_kisim_sureleri.apply(
    lambda row: row["Süre (Saniye)"] / kisim_tezgah_sayilari[row["KISIM"]]
    if row["KISIM"] in kisim_tezgah_sayilari else row["Süre (Saniye)"], 
    axis=1
)

# Diğer kategorisini filtrele
tezgah_basina_kisim_sureleri = tezgah_basina_kisim_sureleri[tezgah_basina_kisim_sureleri['KISIM'] != 'Diğer']
second2minute(tezgah_basina_kisim_sureleri)

# Her bir kısım ve duruş adı için toplam süreleri hesapla
kisim_durus_ozet = latest_week_df.groupby(["KISIM", "Duruş Adı"])["Süre (Saniye)"].sum().reset_index()

# Tezgah bazında toplam bekleme süreleri
latest_week_for_tezgah_sureleri = latest_week_df[latest_week_df['Duruş Adı'] != 'ÇALIŞMA SÜRESİ']
tezgah_sureleri = latest_week_for_tezgah_sureleri.groupby("İş Merkezi Kodu ")["Süre (Saniye)"].sum().reset_index()
second2minute(tezgah_sureleri)
tezgah_sureleri = tezgah_sureleri.sort_values(by="Süre (Dakika)", ascending=True)

# Tezgah ve duruş adına göre grupla
tezgah_durus_ozet = latest_week_df.groupby(["İş Merkezi Kodu ", "Duruş Adı"])["Süre (Saniye)"].sum().reset_index()

# Özel duruşları (yemek, tasarım, SMED) filtrele ve topla
yemek_molasi = tezgah_durus_ozet[tezgah_durus_ozet['Duruş Adı'].str.contains("YEMEK MOLASI", case=False)]
tasarim_durusu = tezgah_durus_ozet[tezgah_durus_ozet['Duruş Adı'].str.contains("TASARIM", case=False)]
smed_durusu = tezgah_durus_ozet[tezgah_durus_ozet['Duruş Adı'].str.contains("SMED", case=False)]

# Gruplara göre topla
yemek_molasi_toplam = yemek_molasi.groupby("İş Merkezi Kodu ")['Süre (Saniye)'].sum().reset_index()
tasarim_durusu_toplam = tasarim_durusu.groupby("İş Merkezi Kodu ")['Süre (Saniye)'].sum().reset_index()
smed_durusu_toplam = smed_durusu.groupby("İş Merkezi Kodu ")['Süre (Saniye)'].sum().reset_index()

# Yeni kategori isimleri ekle
yemek_molasi_toplam['Duruş Adı'] = "YEMEK MOLASI"
tasarim_durusu_toplam['Duruş Adı'] = "TASARIM DURUŞU"
smed_durusu_toplam['Duruş Adı'] = "AYAR"

# Ana veri çerçevesinden ilgili satırları filtrele
tezgah_durus_ozet = tezgah_durus_ozet[~tezgah_durus_ozet['Duruş Adı'].str.contains("YEMEK MOLASI|TASARIM|SMED", case=False)]

# Yeni toplamları birleştir
tezgah_durus_ozet = pd.concat([tezgah_durus_ozet, yemek_molasi_toplam, tasarim_durusu_toplam, smed_durusu_toplam], ignore_index=True)
second2minute(tezgah_durus_ozet)

print("Veri analizi tamamlandı.")

Analiz edilen haftalar: [9, 8, 7, 6]
Haftalık TEE (OEE) değerleri hesaplanıyor...
8. ve 9. hafta görselleri birleştirildi.
Son hafta (9) verileri excel dosyasına kaydedildi.
Toplam duruş süreleri hesaplandı.
Veri analizi tamamlandı.


In [7]:
# Görselleştirme ve Rapor Oluşturma

print("Görselleştirme işlemleri başlatılıyor...")

# Toplam duruş sürelerinin pasta grafiği
visualize(toplam_sureler, threshold=3, baslik="Tüm Tezgahlar Toplam")

# Tezgah başına ortalama duruş süreleri
visualize(tezgah_basina_kisim_sureleri, baslik="Tüm Bölümler (Tezgah Başına)")

# Kısımlara göre duruş sürelerini görselleştirme
for kisim in kisimlar_dict.keys():
    kisim_for_avg = kisim_durus_ozet[kisim_durus_ozet["KISIM"] == kisim].copy()
    
    # Süreler tezgah sayısına bölünüyor
    if len(kisim_for_avg) > 0:
        kisim_for_avg.loc[:, 'Süre (Saniye)'] = kisim_for_avg.apply(
            lambda x: int(x['Süre (Saniye)'] / kisim_tezgah_sayilari[x['KISIM']]), axis=1
        ).astype('int32')
        
        kisim_for_avg = stop_time_sum(kisim_for_avg)
        visualize(kisim_for_avg, baslik=kisim + " (Tezgah Başına)", threshold=3)

# En fazla duruş yapan 10 tezgah
visualize_bar(tezgah_sureleri, "Reds", -10, baslik="En Fazla Duruş Yapan 10 Tezgah")

# En az duruş yapan 10 tezgah
visualize_bar(tezgah_sureleri, "Greens", 0, 10, baslik="En Az Duruş Yapan 10 Tezgah")

# İlk 7 ve son 7 tezgahın karşılaştırması
# Toplam süreyi hesapla
total_time = tezgah_sureleri["Süre (Dakika)"].sum()

# Eşik değeri belirleme
threshold = 0  # Yüzde 0

# Yüzde hesaplama ve eşiği kontrol etme
tezgah_sureleri["Yüzde"] = (tezgah_sureleri["Süre (Dakika)"] / total_time) * 100
filtered_data = tezgah_sureleri[tezgah_sureleri["Yüzde"] > threshold]

# İlk 7 ve son 7 satırı seçme
top = filtered_data.head(7)
bottom = filtered_data.tail(7)

# Seçilen satırları birleştirme
selected_data = pd.concat([top, bottom])

# Renkleri ayarlama: ilk 7 yeşil, son 7 kırmızı
colors = ['#2ca02c'] * len(top) + ['#d62728'] * len(bottom)

# Bar grafiği oluşturma
plt.figure(figsize=(12, 8))

# Bar grafiği çizdir
bars = plt.bar(selected_data["İş Merkezi Kodu "], selected_data["Süre (Dakika)"], color=colors)

# Yüzdeleri her bir barın üstüne ekleme
for bar in bars:
    height = bar.get_height()
    percentage = (height / total_time) * 100  # Yüzde hesaplama
    plt.text(
        bar.get_x() + bar.get_width() / 2, height, 
        f"{height}\n({percentage:.1f}%)",  # Hem süre hem de yüzde gösterme
        ha='center', va='bottom', fontsize=10, color='black', weight='bold'
    )

# Grafik başlık ve eksen etiketleri
plt.title("İş Merkezi Koduna Göre Süre (Dakika) - İlk 7 Yeşil, Son 7 Kırmızı", fontsize=14, pad=15)
plt.xlabel("İş Merkezi Kodu", fontsize=12)
plt.ylabel("Süre (Dakika)", fontsize=12)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig("Raporlar/Genel/İlk 7 ve Son 7 Tezgah.png", dpi=300, bbox_inches='tight')
plt.close()
# Orta bölgedeki tezgahların grafiği
visualize_bar(tezgah_sureleri, bundan=10, buna=-10, text=0)

# Her bir tezgah için duruş süreleri grafiği
print("Tezgah bazında duruş grafikleri oluşturuluyor...")
machine_codes = tezgah_durus_ozet["İş Merkezi Kodu "].unique()
print(f"{len(machine_codes)} tezgah için duruş grafikleri oluşturuluyor...")

# Tüm grafikleri oluştur
create_all_charts(
    tezgah_durus_ozet, 
    machine_column="İş Merkezi Kodu ",
    stoppage_column="Duruş Adı",
    duration_column="Süre (Dakika)",
    threshold=3
)

# 4 haftalık kısım bazında karşılaştırma grafikleri
print("4 haftalık kısım karşılaştırması yapılıyor...")
filtered_kisimlar = filter_sort(four_week_df, max_week=fourth_week)
visualize_2(filtered_kisimlar)

# 4 haftalık tezgah bazında karşılaştırma grafikleri
print("4 haftalık tezgah karşılaştırması yapılıyor...")
filtered_machine = filter_sort(four_week_df, gozlemlenecek='İş Merkezi Kodu ', max_week=fourth_week)
visualize_2(filtered_machine, gozlem="İş Merkezi Kodu ", egiklik=0, palet="Accent")

# İşlem süresini ölç
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Toplam işlem süresi: {elapsed_time:.2f} saniye")

# Tezgah başına düşen işlem süresi
tezgah_sayisi = sum(kisim_tezgah_sayilari.values())
print(f"Toplam tezgah sayısı: {tezgah_sayisi}")
print(f"Tezgah başına işlem süresi: {elapsed_time/tezgah_sayisi:.2f} saniye")

print("Tüm görselleştirme işlemleri tamamlandı!")

Görselleştirme işlemleri başlatılıyor...
Tezgah bazında duruş grafikleri oluşturuluyor...
71 tezgah için duruş grafikleri oluşturuluyor...


C:\Users\ARGESTAJYER\AppData\Local\Temp\3\ipykernel_7156\3145058964.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine_data.loc[:, "Yüzde"] = (machine_data[duration_column] / total_duration) * 100
C:\Users\ARGESTAJYER\AppData\Local\Temp\3\ipykernel_7156\3145058964.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine_data.loc[:, "Yüzde"] = (machine_data[duration_column] / total_duration) * 100
C:\Users\ARGESTAJYER\AppData\Local\Temp\3\ipykernel_7156\3145058964.py:234: SettingWithCopyWarni

Toplam 71 makine için grafikler oluşturuldu
4 haftalık kısım karşılaştırması yapılıyor...
4 haftalık tezgah karşılaştırması yapılıyor...
Toplam işlem süresi: 348.29 saniye
Toplam tezgah sayısı: 71
Tezgah başına işlem süresi: 4.91 saniye
Tüm görselleştirme işlemleri tamamlandı!


In [8]:
# Ana Program - Tüm Analizi Çalıştır

def main():
    """
    Tüm analiz sürecini başlatan ana fonksiyon
    """
    print("Üretim Tezgahları Duruş Analizi Başlatılıyor...")
    
    # Klasör yapısını oluştur
    os.makedirs("Raporlar/Tee/Genel", exist_ok=True)
    os.makedirs("Raporlar/Tee/Kısımlar", exist_ok=True)
    os.makedirs("Raporlar/Tee/Tezgahlar", exist_ok=True)
    os.makedirs("Raporlar/Kısımlar/Son Hafta", exist_ok=True)
    os.makedirs("Raporlar/Kısımlar/Son Hafta Tezgah Başına Ortalama", exist_ok=True)
    os.makedirs("Raporlar/Kısımlar/4 haftalık", exist_ok=True)
    os.makedirs("Raporlar/Tezgahlar/Son Hafta", exist_ok=True)
    os.makedirs("Raporlar/Tezgahlar/4 haftalık", exist_ok=True)
    os.makedirs("Raporlar/Genel", exist_ok=True)
    
    # Diğer hücrelerdeki tüm analizleri çağır
    # Not: Her bir hücre kendi işini yapmaktadır.
    
    print("Analiz tamamlandı! Raporlar klasöründe sonuçları inceleyebilirsiniz.")
    print(f"Toplam işlem süresi: {elapsed_time:.2f} saniye")

if __name__ == "__main__":
    main()

Üretim Tezgahları Duruş Analizi Başlatılıyor...
Analiz tamamlandı! Raporlar klasöründe sonuçları inceleyebilirsiniz.
Toplam işlem süresi: 348.29 saniye
